In [ ]:
import exVAF
va=exVAF.VAF()
import numpy as np
from scipy.optimize import minimize

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

In [ ]:
# calc cost function with inputted corrector kick angles
def cor2bpm(x,ls):
    global sola,solx, ncount
    ncount +=1
    il = len(x)
    xcin = x[0::2]
    ycin = x[1::2]
    for (i,pos) in enumerate(np.array(va.corls)[ls]):
        va.setelem(pos,'theta_x',float(xcin[i]))
        va.setelem(pos+1,'theta_y',float(ycin[i]))
    va.tcs()
    tmp = np.array(va.ldata)
    xbpms = tmp[va.bpmls,1]
    ybpms = tmp[va.bpmls,2]
    r2bpms = np.sqrt(xbpms*xbpms+ybpms*ybpms)
    # calculate cost function
    ans = np.amax(r2bpms) * np.average(r2bpms)*np.std(r2bpms)
    # store best input values
    if ans<sola :
        sola = ans
        solx = x
    return(ans)

# reset corrector parameters
def cor2reset():
    for (i,pos) in enumerate(va.corls[::2]):
        va.setelem(pos,'theta_x',0.0)
        va.setelem(pos+1,'theta_y',0.0)
    va.tcs()

In [ ]:
# Set initial parameter for orbit correction
var = 90 # number of correctors both x and y
ls = range(0,var,2)
x0 = np.zeros(var) # initial corrector kick angles (=0.0)
bnds = np.array([(-3e-3, 3e-3) for _ in range(len(x0))],dtype=float)

In [ ]:
# Run minimization
sola = 1e256 # worst cost value
ncount = 0   # check number of evaluations
res=minimize(cor2bpm,x0,args=ls,method='L-BFGS-B',\
             bounds=bnds,\
             options={'maxiter':10, 'disp':False, 'eps':1.0e-7})

In [ ]:
cor2bpm(solx,ls)
tmp=np.array(va.ldata)
plt.plot(tmp[:,0],tmp[:,1], label='x')
plt.plot(tmp[:,0],tmp[:,2], label='y')
plt.legend(loc='best')
plt.ylabel('orbit [mm]')
plt.xlabel('z [m]')
plt.title('Corrected Orbit')
plt.show()

In [ ]:
cor2reset()
tmp=np.array(va.ldata)
plt.plot(tmp[:,0],tmp[:,1], label='x')
plt.plot(tmp[:,0],tmp[:,2], label='y')
plt.legend(loc='best')
plt.ylabel('orbit [mm]')
plt.xlabel('z [m]')
plt.title('Default Orbit')
plt.show()